In [2]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os
import glob
import cv2
from subprocess import call

from func_viz_flow import viz_flow
from func_OF_util import func_read_flo_file, func_read_pfm_file

import classoftrack

In [29]:
bsize = 10
seqlength=300
startframe=0
#pathl = '/home/kroegert/Documents/VarunSpot/VidExtract/'
#outpath = '/home/kroegert/Documents/VarunSpot/VidExtract_PT/'
#pathl = '/home/kroegert/local/Datasets/GoProWave/GOPR3890/'
#outpath = '/home/kroegert/local/Results/OFPointTrack/'
PFlocal/Datasets/GeoAutomation_Antwerpen/" + seqname + "/images/" + camnp + "/"
outpath = '/home/kroegert/local/Results/OFPointTrack/'
tracksave = outpath + seqname + "_" + camnp + "_tracks.npz"

allimgs = glob.glob(pathl + '*.png')
allimgs.sort()

In [30]:
paramstr = '4 2 16 16 0.05 0.95 0 8 0.50 0 1 1 1 16 13 4.5 2 3 1.6 2 0'

imgtmp = cv2.imread(allimgs[startframe])
imheight = imgtmp.shape[0]
imwidth = imgtmp.shape[1]

oftest = classoftrack.oftrack(bsize, imwidth, imheight, th_flowvalid_ratio = .2, th_flowvalid_abs=1.0)

# keypoint grid (Alternative: detect keypoints in each frame)
xy_l = np.meshgrid(xrange(0,imwidth,20), xrange(0,imheight,20))
xy_l = np.stack((xy_l[0].reshape(-1), xy_l[1].reshape(-1))).T
    
imgs_fl = []
for i in xrange(startframe, startframe+seqlength):
    print "Adding Frame", i
    rndia = np.random.randint(1000)
    rndib = np.random.randint(1000)
    
    resa = call("/home/kroegert/local/Code/OpticalFlow/Github/OF_DIS/build/run_OF_RGB " + allimgs[i]   + " " + allimgs[i+1] + " /tmp/outfile_f" + str(rndia) + ".flo " + paramstr, shell=True)       
    resb = call("/home/kroegert/local/Code/OpticalFlow/Github/OF_DIS/build/run_OF_RGB " + allimgs[i+1] + " " + allimgs[i] + " /tmp/outfile_b" + str(rndib) + ".flo " + paramstr, shell=True)

    call("cp " + allimgs[i  ] + " /tmp/imga.png", shell=True)
    call("cp " + allimgs[i+1] + " /tmp/imgb.png", shell=True)
    resa = call("/home/kroegert/local/Code/OpticalFlow/Github/OF_DIS/build/run_OF_RGB /tmp/imga.png /tmp.imgb.png /tmp/outfile_f" + str(rndia) + ".flo " + paramstr, shell=True)       
    resb = call("/home/kroegert/local/Code/OpticalFlow/Github/OF_DIS/build/run_OF_RGB /tmp/imgb.png /tmp.imga.png /tmp/outfile_b" + str(rndib) + ".flo " + paramstr, shell=True)

    
    print resa, resb
    flowf = func_read_flo_file("/tmp/outfile_f" + str(rndia) + ".flo").astype(float)
    flowb = func_read_flo_file("/tmp/outfile_b" + str(rndib) + ".flo").astype(float)
    
    #imgs_fl.append((flowf, flowb))
    
    imgtmp = cv2.imread(allimgs[i])
    imggray = cv2.cvtColor(imgtmp,cv2.COLOR_BGR2GRAY)
    xy_l= np.copy(cv2.goodFeaturesToTrack(imggray, 1000, 0.001, 5)[:,0,:]) # See opencv documentation for parameters
    #print np.stack((xy[0].reshape(-1), xy[1].reshape(-1))).T

    oftest.addframe(flowf, flowb, xy_l)
    
    #os.system("rm /tmp/outfile_f" + str(rndia) + ".flo")
    #os.system("rm /tmp/outfile_b" + str(rndib) + ".flo")
    
    #x0_plot = viz_flow(flowf[:,:,0], flowf[:,:,1])
    #cv2.imwrite(outpath + '%05i.png' % i, x0_plot)
    
    #plt.figure()
    #plt.imshow(flowf[:,:,1])
    #plt.show()
    
oftest.savetofile(tracksave)

Adding Frame 0
134 134
Adding Frame 1
134 134
Adding Frame 2
134 134
Adding Frame 3
134 134
Adding Frame 4
134 134
Adding Frame 5
134 134
Adding Frame 6
134 134
Adding Frame 7
134 134
Adding Frame 8
134 134
Adding Frame 9
134 134
Adding Frame 10
134 134
Adding Frame 11
134 134
Adding Frame 12
134 134
Adding Frame 13
134 134
Adding Frame 14
134 134
Adding Frame 15
134 134
Adding Frame 16
134 134
Adding Frame 17
134 134
Adding Frame 18
134 134
Adding Frame 19
134 134
Adding Frame 20
134 134
Adding Frame 21
134 134
Adding Frame 22
134 134
Adding Frame 23
134 134
Adding Frame 24
134 134
Adding Frame 25
134 134
Adding Frame 26
134 134
Adding Frame 27
134 134
Adding Frame 28
134 134
Adding Frame 29
134 134
Adding Frame 30
134 134
Adding Frame 31
134 134
Adding Frame 32
134 134
Adding Frame 33
134 134
Adding Frame 34
134 134
Adding Frame 35
134 134
Adding Frame 36
134 134
Adding Frame 37
134 134
Adding Frame 38
134 134
Adding Frame 39
134 134
Adding Frame 40
134 134
Adding Frame 41
134 134
Ad

In [31]:
subsampleno = 1
for fr in xrange(1,seqlength):
    
    print "Plot frame", fr
    xy_t = oftest.getpttransfer(fr, th_min_movement=-1)
    
    #plt.figure(figsize=(16, 12))
    img = cv2.imread(allimgs[startframe + fr])
    img[:] = 255.0
    
    #cv2.line(img, xy_t[0:-1:subsampleno,0,fr], xy_t[0:-1:subsampleno,1,fr], 5)
    #a = np.array([xy_t[0:-1:subsampleno,0,fr]  , xy_t[0:-1:subsampleno,1,fr  ]]).astype(int)
    a = xy_t[0:-1:subsampleno,:,1].astype(int).T
    b = xy_t[0:-1:subsampleno,:,0].astype(int).T
    
    #if (fr>0):
    #   b = np.array([xy_t[0:-1:subsampleno,0,fr-1], xy_t[0:-1:subsampleno,1,fr-1]]).astype(int)
    #    #plt.plot(a, b, color='red')
    #    
    #    a = a.astype(int)
    #    b = b.astype(int)

    #    for lli in xrange(a.shape[1]):
    #        cv2.line(img, (a[0,lli], a[1,lli]), (b[0,lli], b[1,lli]), (0,0,255.0), 1)

    for lli in xrange(a.shape[1]):
        cv2.line(img, (a[0,lli], a[1,lli]), (b[0,lli], b[1,lli]), (0,0,255.0), 2)
            
    #cv2 point plot
    #for lli in xrange(a.shape[1]):
    #    cv2.circle(img, (a[0,lli], a[1,lli]), 3, (0,0,255.0))
        
    imgorg = cv2.imread(allimgs[fr])
    imgorgf = cv2.imread(allimgs[fr+1])
    
    imggray = cv2.cvtColor(imgorg,cv2.COLOR_BGR2GRAY)
    imgdx = imggray[:,0:-1]-imggray[:,1::]
    imgdy = imggray[0:-1,:]-imggray[1::,:]
    
    imgdy = np.minimum(255,np.maximum(0,np.pad(3*imgdy, [(0,1),(0,0)], 'edge')+128))
    imgdy = np.repeat(imgdy[:,:,None],3,2)
    

    imgorgf = cv2.imread(allimgs[fr+1])
    imggrayf = cv2.cvtColor(imgorgf,cv2.COLOR_BGR2GRAY)
    
    imgdt = imggray-imggrayf    
    imgdt = np.minimum(255,np.maximum(0,2*imgdt+128))
    imgdt = np.repeat(imgdt[:,:,None],3,2)
    
    cv2.imwrite(outpath + '%05i.png' % fr, 
                #np.concatenate((
                    np.concatenate((img, imgorg), axis=1)
                 #   ,
                 #              np.concatenate((imgdy, imgdt), axis=1)), 
                 #              axis=0)
               )
    
    #cv2.imwrite(outpath + '%05i.png' % fr, imgdy)
    #plt.imshow(img)
    #plt.hold(True)

Plot frame 1
Plot frame 2
Plot frame 3
Plot frame 4
Plot frame 5
Plot frame 6
Plot frame 7
Plot frame 8
Plot frame 9
Plot frame 10
Plot frame 11
Plot frame 12
Plot frame 13
Plot frame 14
Plot frame 15
Plot frame 16
Plot frame 17
Plot frame 18
Plot frame 19
Plot frame 20
Plot frame 21
Plot frame 22
Plot frame 23
Plot frame 24
Plot frame 25
Plot frame 26
Plot frame 27
Plot frame 28
Plot frame 29
Plot frame 30
Plot frame 31
Plot frame 32
Plot frame 33
Plot frame 34
Plot frame 35
Plot frame 36
Plot frame 37
Plot frame 38
Plot frame 39
Plot frame 40
Plot frame 41
Plot frame 42
Plot frame 43
Plot frame 44
Plot frame 45
Plot frame 46
Plot frame 47
Plot frame 48
Plot frame 49
Plot frame 50
Plot frame 51
Plot frame 52
Plot frame 53
Plot frame 54
Plot frame 55
Plot frame 56
Plot frame 57
Plot frame 58
Plot frame 59
Plot frame 60
Plot frame 61
Plot frame 62
Plot frame 63
Plot frame 64
Plot frame 65
Plot frame 66
Plot frame 67
Plot frame 68
Plot frame 69
Plot frame 70
Plot frame 71
Plot frame 72
P

classoftrack.py:117: RuntimeWarning: invalid value encountered in greater
  idxwasvalid = np.where(self.tracks_valid[fri] & (self.tracks_absmovement[fri] > th_min_movement))[0]


Plot frame 293
Plot frame 294
Plot frame 295
Plot frame 296
Plot frame 297
Plot frame 298
Plot frame 299


In [ ]:
tracks, tracksvalid = oftest.savetofile("outfile")

np.savez_compressed("/tmp/tracksfile.npz", x=tracks)


npzfile = np.load("/tmp/tracksfile.npz")
print npzfile['x'][5][100,:,:].T

#[100,:,:].T

print len(tracks)
print tracks[0].shape
print tracks[5].shape
print tracks[5][100,:,:].T
